In [ ]:
import pandas as pd
from operator import itemgetter

In [ ]:
path2steam_data = "/content/drive/MyDrive/parser_ign/steam_data.csv"
path2yt_data = "/content/drive/MyDrive/parser_ign/youtube_data.csv"

Это IPython Notebook, где я подготавливала CSV файлы.

## Make CSVs

### Functions for preparing CSVs

In [ ]:
def print_valc(tmp):
    for col in tmp.columns:
        print(tmp[col].value_counts())

In [ ]:
def replace_n_drop(df_t, replace_dict={'o': 0, 'о': 0, '1':1,
                                       '0':0, '1.0': 1, '0.0': 0},
                   save_lst=[1, 0, '1.0','0.0', '1', '0'],
                   do_replace=True):
    df = df_t[:]

    # save_lst += [str(i) for i in save_lst if not isinstance(i, str)]

    filter_func = lambda x: x not in save_lst

    if do_replace:
        for col in df.columns:
            df[col] = df[col].replace(list(replace_dict.keys()),
                                                    list(replace_dict.values()))

    for col in df.columns:
            un_vals = list(filter(filter_func, df[col].unique()))
            if len(un_vals) < 5:
                df = df.drop(index=df[ df[col].isin(un_vals) ].index)
                df[col] = df[col].astype(int)

    return df

In [ ]:
def columns_differ(steam_df, youtube_df, display=True):
        cols_steam     = set(steam_df.columns)
        cols_youtube = set(youtube_df.columns)

        common_cols = cols_steam.intersection(cols_youtube)
        different_cols_f = cols_steam.difference(cols_youtube)
        different_cols_s = cols_youtube.difference(cols_steam)

        if display:
            print("Common Columns")
            print(*common_cols, sep='\n')

            print("\nDifferent Columns")
            print('First:')
            print(*different_cols_f, sep='\n')
            print('Second:')
            print(*different_cols_s, sep='\n')
        return different_cols_f, different_cols_s

### Steam data

#### Load data and select columns

In [ ]:
steam_data = pd.read_csv(path2steam_data)
steam_data.head()

,№,Название игры,Кто делал анализ? Фамилия!,Ссылка на Steam/VK Play,Ссылка на игру в VK,Жанр,"""review_score"" (суммарная оценка игры от всех отзывов от 1 до 10)","""review_score_desc"" (суммарная оценка исходя от ""Очень положительные"" или ""Смешанные"")","""total_positive"" (количество положительных отзывов об игре на русском языке)","""total_negative"" (количество негативных отзывов об игре на русском языке)",...,"""timestamp_updated"" (метка времени, когда был последний раз редактирован отзыв)","""votes_up"" (количество пользователей, посчитавших этот отзыв полезным)","""votes_funny"" (количество пользователей, посчитавших этот отзыв забавным)","""weighted_vote_score"" (взвешенная оценка полезности отзыва от 0 до 1 в вещественных числах)","""comment_count"" (число комментариев к отзыву)",Обзор-кратко пересказать с т.з. полит.,Интерпретация игры (о чем она с т.з. полит.?),Кто пишет отзыв? (примерный возрастной интервал),Вывод (по обзорам: какой тренд? по контенту: о чем? общий: сравнение),Примечания
0,1.0,Torn Away,Коркин,https://vkplay.ru/play/game/tornaway/?_1ld=354...,NaN,Приключения,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Война это трагедия. Игра разрушает иллюзии у д...,"Идеологическое противостояния милитаризма, шов...",20-35,Много обзоров. Игра несёт в себе яркий пацифис...,Сразу попался политический обзор
1,2.0,Chernobyl: Origins,Коркин,https://vkplay.ru/play/game/chernobyl_origins/,NaN,Приключения,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Антисоветчины не заметил, достаточно объективн...","1986 год, Чернобыль, происходит катастрофа. Со...",20-35,Дискуссия вокруг наратива игры - антисоветский...,Близость сеттинга игры к каким-то социально зн...
2,3.0,The Tales of Bayun,Андронов,https://vkplay.ru/play/game/bayun/,https://vk.com/kotabauna,Темное фэнтези,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Порадовали славянские сказки и народный фолькл...,Игра погружает в древнерусскую и средневековую...,30-40,"Обзоры имеют 2 основных компонента, первый, по...",NaN
3,4.0,The Mooseman,Иванов,https://vkplay.ru/play/game/mooseman/,NaN,Приключения,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Игра порадовала ориентиром на отечественный а ...,"Игра с этнической мифологией Перми, раскрывает...",30-40,Игра нашла положительный отклик у публики и за...,NaN
4,5.0,FPV Kamikaze Drone,Черкесова,https://vkplay.ru/play/game/fpv_kamikaze_drone/,NaN,Экшен,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Игра ""сырая"",не стоит ее рассматривать как реа...",Военные действия с применением актуальных сейч...,25-40,Немного отзывов. В контексте войны с использов...,NaN


In [ ]:
steam_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2196 entries, 0 to 2195
Data columns (total 79 columns):
 #   Column                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                           --------------  -----  
 0   №                                                                                                                2195 non-null   float64
 1   Название игры                                                                                                    1983 non-null   object 
 2   Кто делал анализ? Фамилия!                                                                                       311 non-null    object 
 3   Ссылка на Steam/VK Play                                                                                          1982 non-null   object 
 4   Ссылка на игру в VK                 

In [ ]:
main_columns = steam_data.columns[14:21]
main_columns

Index(['Обзор (текст: скопировать оригинал): "reviews" (сами отзывы)',
       'Возраст автора (интервалом): До 17 лет; 17-22; 23 и старше; 30+; любой',
       'До 30 (1) или 30+ (0)?', 'Аргументы от ChatGPT',
       'ТИП политического комментария (тема)',
       'Политический ли комментарий? (1/0)',
       'Есть ли в игре общест.-полит. оценка? (1/0)'],
      dtype='object')

In [ ]:
aux_columns = steam_data.columns[22:57]
aux_columns

Index(['Война и мир', 'Игра в войну ', 'Реальность виртуального ',
       'Антимилитаризм', 'Свой-чужой ', 'Патриотизм ',
       'Положительная оценка СССР', 'Отрицательная оценка СССР',
       'Критика 90-ых годов / Все о 90-ых', 'Клюква', 'Политическая ирония',
       'Образ будущего ', 'Повесточка',
       'Запрос на социальную справедливость и равенство',
       'Запрос на объективность ', 'Запрос на альтернативность ',
       'Запрос на субъектность ', 'Негативный национализм',
       'Позитивный национализм ', 'Поддержка СВО', 'Критика СВО',
       'СВО и ее последствия', 'Актуальные проблемы ', 'Критика власти/режима',
       'Поддержка власти/режима', 'Принадлежность к идеологии: левый',
       'Принадлежность к идеологии: правый',
       'Принадлежность к идеологии: либерал/центрист',
       'Ценности: гуманизм/антигуманизм',
       'Мобилизационный и/или мотивационный потенциал ',
       'Международная и/или мировая политика', 'Оппозиция',
       'Личности и\или политики', 'О

In [ ]:
cols = list(itemgetter(*[0, -2, -1])(main_columns))
steam_data[cols]

,"Обзор (текст: скопировать оригинал): ""reviews"" (сами отзывы)",Политический ли комментарий? (1/0),Есть ли в игре общест.-полит. оценка? (1/0)
0,"Одна из тех не многих игр, по прохождению кото...",1,1.0
1,"Узнал , про эту игру из ленты новостей. Я как ...",1,0.0
2,Прошел я Tales of Bayun от российской студии F...,1,0.0
3,"Купил эту игру из подборки, в порыве поддержат...",1,0.0
4,"По совету проверенного товарища, третьего дня ...",1,1.0
...,...,...,...
2191,"Игра хорошая, но покупать никому не советую. А...",1,0.0
2192,"ИИ союзников всратый. Ты крадёшься, а он встаё...",0,0.0
2193,Довольно занятная игра (стратегия и ролевая) п...,0,0.0
2194,Вы никого не сможете убить в этой игре по нача...,0,0.0


#### DataFrame for binary classification

In [ ]:
for col in cols[1:]:
    print(steam_data[col].value_counts())

Политический ли комментарий? (1/0)
1       1301
0        891
А          1
укр        1
1301       1
Name: count, dtype: int64
Есть ли в игре общест.-полит. оценка? (1/0)
1.0    1247
0.0     946
Name: count, dtype: int64
№                                                                           1
Название игры                                                             213
Кто делал анализ? Фамилия!                                               1885
Ссылка на Steam/VK Play                                                   214
Ссылка на игру в VK                                                      2195
                                                                         ... 
Обзор-кратко пересказать с т.з. полит.                                   1907
Интерпретация игры (о чем она с т.з. полит.?)                            2046
Кто пишет отзыв? (примерный возрастной интервал)                         1888
Вывод (по обзорам: какой тренд? по контенту: о чем? общий: сравнение)    2142


In [ ]:
bin_train_df = steam_data[cols].rename(columns=dict(zip(cols, ['text', 'label', 'assessment'])))
print(bin_train_df.isna().sum())

text          2
label         1
assessment    3
dtype: int64


In [ ]:
bin_train_df = bin_train_df.dropna()
print(bin_train_df.isna().sum())

text          0
label         0
assessment    0
dtype: int64


In [ ]:
for col in bin_train_df.columns:
    print(bin_train_df[col].value_counts())

text
Игра хорошая, но покупать никому не советую. Автор заявил, что весь доход, полученной от Российской Федерации направит на Украину. Учитывая, что автор игры не отправляет доход от США, убившей сотни тысяч людей по всему миру, в пострадавшие страны, это мерзость и лицемерие.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [ ]:
bin_train_df.loc[bin_train_df.label == 'укр', 'label'] = 1

In [ ]:
bin_train_df = bin_train_df.drop(index=bin_train_df[bin_train_df.label == 'А'].index)

In [ ]:
bin_train_df.label = bin_train_df.label.astype(float)

In [ ]:
bin_train_df.to_csv("./steam_df_bin_clf.csv")

#### DataFrame for multilabel classification

In [ ]:
steam_data[list(cols) + list(aux_columns)].isna().sum()

Обзор (текст: скопировать оригинал): "reviews" (сами отзывы)    2
Политический ли комментарий? (1/0)                              1
Есть ли в игре общест.-полит. оценка? (1/0)                     3
Война и мир                                                     6
Игра в войну                                                    6
Реальность виртуального                                         6
Антимилитаризм                                                  6
Свой-чужой                                                      6
Патриотизм                                                      6
Положительная оценка СССР                                       6
Отрицательная оценка СССР                                       6
Критика 90-ых годов / Все о 90-ых                               6
Клюква                                                          6
Политическая ирония                                             7
Образ будущего                                                  6
Повесточка

In [ ]:
for col in aux_columns:
    print(steam_data[col].value_counts())

Война и мир
0.0      1593
1.0       596
596.0       1
Name: count, dtype: int64
Игра в войну 
0.0      1906
1.0       283
283.0       1
Name: count, dtype: int64
Реальность виртуального 
0      1748
1       440
й         1
440       1
Name: count, dtype: int64
Антимилитаризм
0.0      2003
1.0       186
186.0       1
Name: count, dtype: int64
Свой-чужой 
0      1962
1       226
й         1
226       1
Name: count, dtype: int64
Патриотизм 
0.0      2002
1.0       187
187.0       1
Name: count, dtype: int64
Положительная оценка СССР
0.0     2132
1.0       57
57.0       1
Name: count, dtype: int64
Отрицательная оценка СССР
0.0     2166
1.0       23
23.0       1
Name: count, dtype: int64
Критика 90-ых годов / Все о 90-ых
0.0     2174
1.0       15
15.0       1
Name: count, dtype: int64
Клюква
0.0      2089
1.0       100
100.0       1
Name: count, dtype: int64
Политическая ирония
0      1925
1       262
й         1
262       1
Name: count, dtype: int64
Образ будущего 
0.0     2162
1.0       2

In [ ]:
train_df_mulicls = steam_data[list(cols) + list(aux_columns)].dropna()

In [ ]:
tmp = replace_n_drop(train_df_mulicls)

In [ ]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2177 entries, 0 to 2194
Data columns (total 38 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Обзор (текст: скопировать оригинал): "reviews" (сами отзывы)  2177 non-null   object 
 1   Политический ли комментарий? (1/0)                            2177 non-null   float64
 2   Есть ли в игре общест.-полит. оценка? (1/0)                   2177 non-null   float64
 3   Война и мир                                                   2177 non-null   float64
 4   Игра в войну                                                  2177 non-null   float64
 5   Реальность виртуального                                       2177 non-null   float64
 6   Антимилитаризм                                                2177 non-null   float64
 7   Свой-чужой                                                    2177 non-nul

In [ ]:
train_df_mulicls.to_csv("./steam_df_multi_clf.csv")

### YouTube Data

In [ ]:
yt_data = pd.read_csv(path2yt_data)
yt_data.head()

<ipython-input-132-05c35ec4ace9>:1: DtypeWarning: Columns (2,3,4,5,6,9,10,11,12,13,14,18,20,21,22,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  yt_data = pd.read_csv(path2yt_data)


,№,Название видео,"Кто делал анализ? Фамилия, пароли, явки!",Ссылка Youtube,Дата,Просмотры,Лайки,Комментарии,Всего: П+Л+К,% П от ПЛК,...,Личности и\или политики,Образ народа,Иное,"ТЭГИ ИНОГО (Темы, которые не были выделены отдельно!)",Кол-во ТИПОВ,Комментарий-кратко пересказать с т.з. полит.,Интерпретация видео (о чем оно с т.з. полит.?),Кто пишет отзыв? (примерный возрастной интервал),Вывод (по комментариям: какой тренд? по контенту: о чем? общий: сравнение),Примечания
0,1.0,СССР ИЗ БУДУЩЕГО ► Atomic Heart #1,Андронов,https://www.youtube.com/watch?v=VuLaqbxq5zA,26.01.2024,13167982,625000,17664.0,13810646.0,95,...,0.0,0.0,0.0,NaN,1.0,Автор сравнивает игру с другими играми в разли...,"Прохождение, в котором рассказывается основной...",35-55,"Судя по прохождению можно сказать, что обзорщи...",NaN
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,3.0,Автор сравнивает детализованность звука сирены...,NaN,18-30,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,1.0,Автор пишет о приятной ностальгии по СССР.,NaN,25-35,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,1.0,Автор пишет про идеальный образ СССР.,NaN,25-35,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,2.0,Автор высказывает теорию о схожести событий иг...,NaN,25-35,NaN,NaN


In [ ]:
yt_data.columns

Index(['№', 'Название видео', 'Кто делал анализ? Фамилия, пароли, явки!',
       'Ссылка Youtube', 'Дата', 'Просмотры', 'Лайки', 'Комментарии',
       'Всего: П+Л+К', '% П от ПЛК', '% Л от ПЛК', '% К от ПЛК',
       'Категория (Фильмы, Игры, Музыка, Оппозиционеры)',
       'Временной интервал (с 01.01.23 по 31.12.23): указывать интервал',
       'Тема, к которой относится видео, что было в Google-трендах (перечисление; поиск по фильтрам)',
       'Кол-во ответов на комментарий', 'id канала',
       'Комментарий (текст: скопировать оригинал)',
       'Возраст автора (интервалом): До 17 лет; 17-22; 23 и старше; 30+; любой',
       'До 30 (1) или 30+ (0)?', 'Аргументы от ChatGPT', 'Лайки комментария',
       'ТИП политического комментария (тема)',
       'Политический ли комментарий? (1/0)',
       'Есть ли в видео общест.-полит. оценка? (1/0)',
       'Тэги по видео (по описанию: ключ. слова, только ПОЛИТ СОСТАВЛ)',
       'Война и мир', 'Игра в войну ', 'Реальность виртуального ',
     

In [ ]:
yt_cols_main = [ 'Комментарий (текст: скопировать оригинал)',
                                'Политический ли комментарий? (1/0)',
                                'Есть ли в видео общест.-полит. оценка? (1/0)'
                ]

In [ ]:
len(yt_data)

45571

In [ ]:
yt_data[yt_cols_main].isna().sum()

Комментарий (текст: скопировать оригинал)           3
Политический ли комментарий? (1/0)              35970
Есть ли в видео общест.-полит. оценка? (1/0)    35975
dtype: int64

In [ ]:
yt_data[yt_cols_main +  list(aux_columns)].isna().sum()

Комментарий (текст: скопировать оригинал)              3
Политический ли комментарий? (1/0)                 35970
Есть ли в видео общест.-полит. оценка? (1/0)       35975
Война и мир                                        35980
Игра в войну                                       35981
Реальность виртуального                            35977
Антимилитаризм                                     35978
Свой-чужой                                         35978
Патриотизм                                         35976
Положительная оценка СССР                          35979
Отрицательная оценка СССР                          35980
Критика 90-ых годов / Все о 90-ых                  35978
Клюква                                             35978
Политическая ирония                                35980
Образ будущего                                     35977
Повесточка                                         35978
Запрос на социальную справедливость и равенство    35977
Запрос на объективность        

In [ ]:
yt_bin_data = yt_data[yt_cols_main].dropna()

In [ ]:
yt_multi_clf_data = yt_data[yt_cols_main +  list(aux_columns)].dropna()

In [ ]:
print_valc(yt_bin_data)

Комментарий (текст: скопировать оригинал)
Слава Украине                                                                                                                                           8
Слава России                                                                                                                                            7
Нужно отправить назад тех кто не знает русских язык, не знает историю России и культуры. Кто совершил убийство или насилие.                             6
Слава Україні                                                                                                                                           6
Слава России и Путину, Слава ВС РФ, Слава ЧВК Вагнер, Слава героям России и СССР 🇷🇺🇷🇺☦️💖⭐🎶✌️🤟❤🤞♥🤙☝️☀️🤛🤛🔥🇷🇺🇷🇺🇷🇺💪😎😎😎                                      5
                                                                                                                                                       ..
Войне не Россия ааа СССР был      

In [ ]:
print_valc(yt_multi_clf_data)

Комментарий (текст: скопировать оригинал)
Слава Украине                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
yt_multi_clf_df = replace_n_drop(yt_multi_clf_data, do_replace=False)

save data

In [ ]:
yt_bin_data.to_csv('./youtube_df_bin_clf.csv')

In [ ]:
yt_multi_clf_df.to_csv('./youtube_df_multi_clf.csv')

### Merge data and save

#### Data for Binary Classification

In [ ]:
steam_df     = pd.read_csv('/content/steam_df_bin_clf.csv', index_col=0)
youtube_df = pd.read_csv('/content/youtube_df_bin_clf.csv', index_col=0)

cols_steam     = set(steam_df.columns)
cols_youtube = set(youtube_df.columns)

cols_steam, cols_youtube

({'assessment', 'label', 'text'},
 {'Есть ли в видео общест.-полит. оценка? (1/0)',
  'Комментарий (текст: скопировать оригинал)',
  'Политический ли комментарий? (1/0)'})

In [ ]:
youtube_df = youtube_df.rename(columns={'Есть ли в видео общест.-полит. оценка? (1/0)' : 'assessment',
                                'Комментарий (текст: скопировать оригинал)' : 'text',
                                 'Политический ли комментарий? (1/0)' : 'label'})

In [ ]:
cols_steam     = set(steam_df.columns)
cols_youtube = set(youtube_df.columns)

common_cols = cols_steam.intersection(cols_youtube)
different_cols = cols_steam.difference(cols_youtube)

print("Common Columns")
print(*common_cols, sep='\n')

print("\nDifferent Columns")
print(*different_cols, sep='\n')

Common Columns
label
assessment
text

Different Columns



In [ ]:
joined_df = pd.concat([steam_df, youtube_df], ignore_index=True)

In [ ]:
joined_df

,text,label,assessment
0,"Одна из тех не многих игр, по прохождению кото...",1.0,1.0
1,"Узнал , про эту игру из ленты новостей. Я как ...",1.0,0.0
2,Прошел я Tales of Bayun от российской студии F...,1.0,0.0
3,"Купил эту игру из подборки, в порыве поддержат...",1.0,0.0
4,"По совету проверенного товарища, третьего дня ...",1.0,1.0
...,...,...,...
11774,Не ужели вы не можете понять что это капкан дл...,1.0,1.0
11775,Это Божественное наказание евреям за поддержку...,1.0,1.0
11776,Израиль заслужил,1.0,1.0
11777,Имеется очень простая формула: мир × производс...,1.0,1.0


In [ ]:
joined_df.to_csv('./joined_df_bin_clf.csv')

#### Data for Multi-label Classification

In [ ]:
steam_df     = pd.read_csv('/content/multi_steam_df.csv')
youtube_df = pd.read_csv('/content/multi_yt_df.csv')

<ipython-input-4-3fc9a6d77297>:2: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  youtube_df = pd.read_csv('/content/multi_yt_df.csv')


In [ ]:
f, s = columns_differ(steam_df, youtube_df)

Common Columns
Реальность виртуального 
Свой-чужой 
assessment
СВО и ее последствия
Международная и/или мировая политика
Принадлежность к идеологии: левый
Оппозиция
text
Игра в войну 
Позитивный национализм 
Ценности: гуманизм/антигуманизм
label
Политическая ирония
Положительная оценка СССР
Патриотизм 
Принадлежность к идеологии: правый
Запрос на субъектность 
Поддержка СВО
Запрос на альтернативность 
Отрицательная оценка СССР
Критика СВО
Критика 90-ых годов / Все о 90-ых
Запрос на социальную справедливость и равенство
Запрос на объективность 
Образ будущего 
Повесточка
Актуальные проблемы 
Клюква
Негативный национализм
Принадлежность к идеологии: либерал/центрист
Война и мир
Поддержка власти/режима
age
Критика власти/режима
Антимилитаризм
Мобилизационный и/или мотивационный потенциал 
Личности и\или политики

Different Columns
First:

Second:



In [ ]:
steam_df = steam_df.drop(columns=list(f))
youtube_df = youtube_df.drop(columns=list(s))

In [ ]:
dict_names = {'Есть ли в игре общест.-полит. оценка? (1/0)' : 'Есть ли в видео общест.-полит. оценка? (1/0)',
                          'Обзор (текст: скопировать оригинал): "reviews" (сами отзывы)' : 'Комментарий (текст: скопировать оригинал)'
              }

In [ ]:
steam_df = steam_df.rename(columns=dict_names)

In [ ]:
dict_names = columns_differ(steam_df, youtube_df, display=False)
dict_names

(set(), set())

In [ ]:
joined_df = pd.concat([steam_df, youtube_df], ignore_index=True)

In [ ]:
joined_df.head()

,text,age,label,assessment,Война и мир,Игра в войну,Реальность виртуального,Антимилитаризм,Свой-чужой,Патриотизм,...,Критика власти/режима,Поддержка власти/режима,Принадлежность к идеологии: левый,Принадлежность к идеологии: правый,Принадлежность к идеологии: либерал/центрист,Ценности: гуманизм/антигуманизм,Мобилизационный и/или мотивационный потенциал,Международная и/или мировая политика,Оппозиция,Личности и\или политики
0,"Одна из тех не многих игр, по прохождению кото...",23-30,1.0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,NaN,6
1,"Узнал , про эту игру из ленты новостей. Я как ...",17-22,0.0,0,0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NaN,3
2,Прошел я Tales of Bayun от российской студии F...,23-30,0.0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NaN,3
3,"Купил эту игру из подборки, в порыве поддержат...",30+,0.0,0,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0,0,NaN,4
4,"По совету проверенного товарища, третьего дня ...",23-30,1.0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0,0,NaN,6


In [ ]:
joined_df = replace_n_drop(joined_df, do_replace=True)

In [ ]:
print_valc(joined_df)

text
Слава Украине                                                                                                                                                                                                                   10
Слава Україні                                                                                                                                                                                                                    8
Нужно отправить назад тех кто не знает русских язык, не знает историю России и культуры. Кто совершил убийство или насилие.                                                                                                      6
Слава России                                                                                                                                                                                                                     6
я гей                                                                                  

In [ ]:
for col in joined_df.columns:
    if len(joined_df[col].unique()) > 2:
        print([col])

['text']
['age']
['assessment']


In [ ]:
joined_df['Война и мир'] = joined_df['Война и мир'].fillna(0)

In [ ]:
joined_df['Война и мир'].unique()

array([1, 0, nan], dtype=object)

In [ ]:
joined_df = joined_df.drop(columns=['Оппозиция', 'Личности и\\или политики'])

In [ ]:
joined_df.columns

Index(['text', 'age', 'label', 'assessment', 'Война и мир', 'Игра в войну ',
       'Реальность виртуального ', 'Антимилитаризм', 'Свой-чужой ',
       'Патриотизм ', 'Положительная оценка СССР', 'Отрицательная оценка СССР',
       'Критика 90-ых годов / Все о 90-ых', 'Клюква', 'Политическая ирония',
       'Образ будущего ', 'Повесточка',
       'Запрос на социальную справедливость и равенство',
       'Запрос на объективность ', 'Запрос на альтернативность ',
       'Запрос на субъектность ', 'Негативный национализм',
       'Позитивный национализм ', 'Поддержка СВО', 'Критика СВО',
       'СВО и ее последствия', 'Актуальные проблемы ', 'Критика власти/режима',
       'Поддержка власти/режима', 'Принадлежность к идеологии: левый',
       'Принадлежность к идеологии: правый',
       'Принадлежность к идеологии: либерал/центрист',
       'Ценности: гуманизм/антигуманизм',
       'Мобилизационный и/или мотивационный потенциал ',
       'Международная и/или мировая политика', 'Оппозиция'

In [ ]:
print_valc(joined_df)

text
Слава Украине                                                                                                                                                                                                                   10
Слава Україні                                                                                                                                                                                                                    8
Нужно отправить назад тех кто не знает русских язык, не знает историю России и культуры. Кто совершил убийство или насилие.                                                                                                      6
Слава России                                                                                                                                                                                                                     6
я гей                                                                                  

In [ ]:
joined_df.to_csv('./joined_df_multi_clean.csv')

In [ ]:
names = {'Комментарий (текст: скопировать оригинал)' : 'text',
                 'Политический ли комментарий? (1/0)' : 'label',
                 'Есть ли в видео общест.-полит. оценка? (1/0)' : 'assessment'
}

In [ ]:
joined_df = joined_df.rename(columns=names)

In [ ]:
joined_df.to_csv('./joined_df_multi_clf_eng.csv')